In [1]:
import librosa
import numpy as np
import os
import math
import operator
from sklearn.cluster import KMeans
import hmmlearn.hmm
import functionMFCC as func

C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [25]:
class_names = ["nguoi", "viet_nam", "nha", "cua", "khong", "test_nguoi", "test_viet_nam", "test_nha", "test_cua", "test_khong"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = func.get_class_data(os.path.join("data_hmm", cname))
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
# print("vectors", all_vectors.shape)
# # Run K-Means algorithm to get clusters
# kmeans = func.clustering(all_vectors)
# print("centers", kmeans.cluster_centers_.shape)
# for cname in class_names:
#     dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

Load nguoi dataset
Load viet_nam dataset
Load nha dataset
Load cua dataset
Load khong dataset
Load test_nguoi dataset
Load test_viet_nam dataset
Load test_nha dataset
Load test_cua dataset
Load test_khong dataset


In [26]:
def train_model(train_set, n_component):
    model = hmmlearn.hmm.GMMHMM(n_components=n_component, n_mix=3, n_iter=1000, params='mtcw', init_params='mst', verbose=True)
    X = np.concatenate(train_set)
    lengths = list([len(x) for x in train_set])
    # fit dat 
    print(X.shape, lengths, len(lengths))
    model.fit(X)
    return model

In [32]:
# startprob9 = np.array([0.7, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# startprob18 = np.array([0.7, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# transmat6 = np.array([[0.05 for i in range(6)] for j in range(6)])
# transmat12 = np.array([[0.05 for i in range(12)] for j in range(12)])
# for x in range(5):
#     transmat6[x][x] = 0.4
#     transmat6[x][x+1] = 0.4
# transmat6[5][5] = 0.75
# for x in range(11):
#     transmat12[x][x] = 0.25
#     transmat12[x][x+1] = 0.25
# transmat12[11][11] = 0.45
# print(transmat6)
models = {}
models["khong"] = train_model(dataset["khong"], 9)
models["nha"] = train_model(dataset["nha"], 9)
models["cua"] = train_model(dataset["cua"], 9)
models["nguoi"] = train_model(dataset["nguoi"], 9)
models["viet_nam"] = train_model(dataset["viet_nam"], 18)

(1887, 36) [25, 21, 24, 27, 25, 21, 18, 24, 24, 21, 24, 29, 15, 31, 19, 21, 20, 25, 29, 32, 31, 23, 18, 35, 23, 21, 24, 28, 33, 28, 34, 23, 17, 17, 17, 23, 25, 25, 22, 25, 28, 22, 24, 19, 17, 21, 19, 23, 19, 35, 26, 24, 27, 21, 20, 23, 20, 21, 24, 16, 25, 19, 26, 21, 22, 26, 27, 19, 17, 35, 28, 22, 25, 29, 31, 21, 19, 21, 13, 25] 80


         1     -193782.4546             +nan
         2     -179374.0205      +14408.4341
         3     -177114.5577       +2259.4629
         4     -176505.5536        +609.0041
         5     -176238.7590        +266.7945
         6     -176091.3690        +147.3900
         7     -176000.9488         +90.4202
         8     -175917.2639         +83.6850
         9     -175843.3529         +73.9110
        10     -175799.8800         +43.4728
        11     -175747.6063         +52.2737
        12     -175685.8889         +61.7174
        13     -175647.6760         +38.2128
        14     -175616.3100         +31.3660
        15     -175566.8661         +49.4439
        16     -175489.3704         +77.4957
        17     -175474.4181         +14.9523
        18     -175460.9058         +13.5124
        19     -175452.0118          +8.8940
        20     -175446.8388          +5.1729
        21     -175441.3332          +5.5057
        22     -175430.9613         +10.3719
        23

(2541, 36) [21, 22, 30, 23, 17, 23, 28, 47, 32, 24, 42, 32, 33, 36, 25, 20, 25, 45, 22, 31, 28, 17, 42, 27, 19, 20, 33, 16, 36, 31, 39, 17, 33, 31, 28, 35, 32, 38, 18, 43, 29, 25, 22, 25, 21, 24, 32, 23, 62, 33, 22, 18, 33, 22, 16, 18, 54, 46, 52, 45, 22, 40, 32, 25, 26, 41, 32, 43, 43, 43, 26, 59, 40, 45, 32, 50, 47, 59, 32, 21] 80


         1     -245280.0540             +nan
         2     -225254.1917      +20025.8623
         3     -193456.0032      +31798.1886
         4     -181309.7051      +12146.2980
         5     -180929.1124        +380.5927
         6     -180736.5366        +192.5758
         7     -180591.3295        +145.2072
         8     -180461.8054        +129.5240
         9     -180399.2913         +62.5142
        10     -180371.8972         +27.3940
        11     -180350.8786         +21.0187
        12     -180333.7017         +17.1768
        13     -180315.6249         +18.0769
        14     -180296.5464         +19.0785
        15     -180278.0895         +18.4569
        16     -180256.9364         +21.1531
        17     -180239.9845         +16.9519
        18     -180230.0189          +9.9656
        19     -180222.8634          +7.1555
        20     -180215.4861          +7.3774
        21     -180207.3113          +8.1748
        22     -180196.3526         +10.9587
        23

(1854, 36) [18, 14, 29, 24, 21, 18, 18, 15, 18, 13, 17, 23, 14, 16, 18, 18, 16, 20, 15, 16, 24, 18, 17, 25, 21, 24, 23, 19, 24, 24, 17, 19, 18, 24, 23, 18, 18, 20, 19, 20, 16, 15, 17, 18, 19, 22, 32, 21, 25, 21, 25, 20, 20, 14, 17, 24, 18, 15, 22, 16, 16, 15, 19, 16, 33, 22, 19, 19, 16, 20, 18, 23, 14, 17, 20, 14, 17, 11, 19, 23, 17, 16, 16, 20, 24, 15, 17, 20, 19, 23, 21, 22, 15, 26, 19, 20] 96


         1     -200057.3749             +nan
         2     -189635.8093      +10421.5657
         3     -187379.3617       +2256.4476
         4     -186822.9022        +556.4595
         5     -186638.1515        +184.7507
         6     -186531.0463        +107.1052
         7     -186428.3533        +102.6930
         8     -186342.4491         +85.9042
         9     -186277.8345         +64.6146
        10     -186242.0150         +35.8196
        11     -186220.3903         +21.6247
        12     -186186.1526         +34.2376
        13     -186167.6067         +18.5460
        14     -186153.8438         +13.7629
        15     -186142.2812         +11.5626
        16     -186127.6690         +14.6122
        17     -186108.0552         +19.6138
        18     -186092.4088         +15.6464
        19     -186084.4258          +7.9830
        20     -186080.2816          +4.1442
        21     -186076.2654          +4.0162
        22     -186072.3926          +3.8728
        23

(2020, 36) [29, 30, 15, 43, 30, 30, 14, 25, 24, 30, 35, 23, 17, 24, 21, 31, 22, 28, 29, 28, 19, 16, 16, 31, 24, 22, 19, 13, 21, 23, 30, 19, 32, 47, 18, 34, 26, 21, 10, 23, 29, 25, 21, 25, 25, 12, 27, 24, 37, 22, 26, 29, 17, 26, 24, 33, 36, 22, 21, 30, 31, 31, 22, 22, 26, 23, 36, 32, 23, 17, 32, 28, 21, 32, 21, 16, 25, 26, 35, 18] 80


         1     -211962.0511             +nan
         2     -198429.4025      +13532.6486
         3     -196183.0358       +2246.3667
         4     -195609.2905        +573.7454
         5     -195324.2079        +285.0825
         6     -195175.5064        +148.7015
         7     -195051.7781        +123.7284
         8     -194951.1900        +100.5881
         9     -194889.6003         +61.5896
        10     -194815.9139         +73.6865
        11     -194790.9878         +24.9260
        12     -194778.2222         +12.7656
        13     -194755.0924         +23.1298
        14     -194724.9081         +30.1843
        15     -194679.5176         +45.3905
        16     -194607.9049         +71.6127
        17     -194590.8855         +17.0194
        18     -194573.2403         +17.6451
        19     -194556.3781         +16.8622
        20     -194544.5647         +11.8134
        21     -194537.0500          +7.5146
        22     -194530.3655          +6.6845
        23

(3536, 36) [52, 43, 35, 55, 34, 55, 47, 51, 41, 49, 34, 58, 41, 41, 75, 30, 50, 53, 39, 42, 40, 42, 49, 40, 39, 36, 40, 41, 54, 47, 46, 55, 38, 49, 37, 35, 45, 39, 58, 42, 40, 43, 34, 31, 47, 48, 37, 58, 38, 37, 50, 41, 37, 43, 50, 49, 48, 44, 51, 32, 42, 51, 42, 37, 30, 50, 45, 35, 39, 55, 47, 45, 39, 40, 61, 56, 42, 40, 46, 39] 80


         1     -365751.4077             +nan
         2     -333119.0201      +32632.3876
         3     -328668.4611       +4450.5590
         4     -327119.6087       +1548.8524
         5     -326435.3103        +684.2984
         6     -325984.0295        +451.2808
         7     -325775.4462        +208.5833
         8     -325574.4733        +200.9729
         9     -325390.3955        +184.0778
        10     -325211.7220        +178.6735
        11     -325001.6265        +210.0955
        12     -324819.6694        +181.9572
        13     -324715.6341        +104.0352
        14     -324640.2346         +75.3995
        15     -324568.5124         +71.7222
        16     -324505.8208         +62.6915
        17     -324467.7437         +38.0771
        18     -324429.2077         +38.5359
        19     -324387.9428         +41.2650
        20     -324348.9921         +38.9507
        21     -324316.3647         +32.6274
        22     -324283.6227         +32.7419
        23

In [33]:
print("Testing")
for true_cname in class_names:
    if true_cname[:4] == "test":
        result = 0
        length = 0
        for O in dataset[true_cname]:
            length+=1
            score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
#             print(true_cname, score, max(score.items(), key=operator.itemgetter(1))[0])
            if true_cname[5:] == max(score.items(), key=operator.itemgetter(1))[0]:
                result = result + 1
        print('Acc ' + true_cname[5:] , result/length, result, length)

Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Testing


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc nguoi 0.9 18 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


...

Acc nha 0.95 19 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc cua 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc khong 1.0 20 20


In [18]:
import pickle

In [34]:
with open("modelHMM.pkl", "wb") as file:
    pickle.dump(models, file)